<a href="https://colab.research.google.com/github/musa1993/B/blob/master/Copy_of_amazon_deforestation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#Reading the train classes that contains the image names and tags from the directory
train_classes = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
train_classes.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [ ]:
labels = set()
def splitting_tags(tags):
    '''
    Takes in tags column, splits the tags and store as a set
    '''
    [labels.add(tag) for tag in tags.split()]
    
# Create a copy of train_classes
train_classes1 = train_classes.copy()
train_classes1['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)

['clear', 'blow_down', 'agriculture', 'road', 'slash_burn', 'cloudy', 'water', 'cultivation', 'selective_logging', 'artisinal_mine', 'partly_cloudy', 'conventional_mine', 'primary', 'blooming', 'haze', 'habitation', 'bare_ground']


In [ ]:
#confirm that the length of the dataframe is the same as the shape
assert len(train_classes1['image_name'].unique()) == train_classes1.shape[0]

In [ ]:
##One hot encoding is performed on the labels in train classes 

for tag in labels:
    train_classes1[tag] = train_classes1['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
## adding .jpg extension to the column image_name so as to have same name format as the image files
train_classes1['image_name'] = train_classes1['image_name'].apply(lambda x: '{}.jpg'.format(x))
train_classes1.head()

,image_name,tags,clear,blow_down,agriculture,road,slash_burn,cloudy,water,cultivation,selective_logging,artisinal_mine,partly_cloudy,conventional_mine,primary,blooming,haze,habitation,bare_ground
0,train_0.jpg,haze primary,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,train_1.jpg,agriculture clear primary water,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,train_2.jpg,clear primary,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,train_3.jpg,clear primary,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0


In [ ]:
#importing libraries for training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Defining the columns,i.e the labels that were newly added to the train_classes via hot encoding.
columns = list(train_classes1.columns[2:])

In [ ]:
columns

['clear',
 'blow_down',
 'agriculture',
 'road',
 'slash_burn',
 'cloudy',
 'water',
 'cultivation',
 'selective_logging',
 'artisinal_mine',
 'partly_cloudy',
 'conventional_mine',
 'primary',
 'blooming',
 'haze',
 'habitation',
 'bare_ground']

In [ ]:
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    '''
    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        beta = 2
        epsilon= 1e-4
        
    Returns:
        fbeta score
    '''
    
    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [ ]:
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    '''
    Retuns accuracy value for multi_label classification
    
    Set y_true and y_pred

    Args:
        y_true: correct target values
        Y_pred: predicted values returned by the classifer
        epsilon= 1e-4
        
    Returns:
        Accuracy score
    '''
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32)
                       * tf.cast(tf.logical_not(y_pred), tf.float32), axis = 1)
    
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [ ]:
#defining our model
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(lr=1e-4)
    
    # We need binary here, since categorical_crossentropy l1 norms the output before calculating loss.
    model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [ ]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', 
                                        monitor = 'val_fbeta',
                                        mode = 'max',
                                        save_best_only = True,
                                        save_weights_only = True)

In [ ]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="../input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="../input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [ ]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [ ]:
#initialize the model
model1 = build_model()

In [ ]:
# Preview the model architecture
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 64)        3

In [ ]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, 
           steps_per_epoch = step_train_size, 
           validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 25, 
           callbacks = [save_best_check_point])

Epoch 1/25
2024/2024 [==============================] - 175s 87ms/step - loss: 0.2170 - multi_label_acc: 0.9175 - fbeta: 0.7037 - val_loss: 0.1595 - val_multi_label_acc: 0.9374 - val_fbeta: 0.7822
Epoch 2/25
2024/2024 [==============================] - 116s 57ms/step - loss: 0.1622 - multi_label_acc: 0.9356 - fbeta: 0.7840 - val_loss: 0.1417 - val_multi_label_acc: 0.9440 - val_fbeta: 0.8070
Epoch 3/25
2024/2024 [==============================] - 116s 57ms/step - loss: 0.1479 - multi_label_acc: 0.9409 - fbeta: 0.8069 - val_loss: 0.1317 - val_multi_label_acc: 0.9468 - val_fbeta: 0.8189
Epoch 4/25
2024/2024 [==============================] - 117s 58ms/step - loss: 0.1383 - multi_label_acc: 0.9450 - fbeta: 0.8230 - val_loss: 0.1274 - val_multi_label_acc: 0.9498 - val_fbeta: 0.8407
Epoch 5/25
2024/2024 [==============================] - 115s 57ms/step - loss: 0.1329 - multi_label_acc: 0.9473 - fbeta: 0.8319 - val_loss: 0.1222 - val_multi_label_acc: 0.9510 - val_fbeta: 0.8460
Epoch 6/25
2024

In [ ]:
#initializing a second model to make predictions
model2 = build_model()

In [ ]:
##adding .jpg extension to image name in the sample submission file
sample_submission = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission1 = sample_submission.copy()
sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission1.head()

,image_name,tags
0,test_0.jpg,primary clear agriculture road water
1,test_1.jpg,primary clear agriculture road water
2,test_2.jpg,primary clear agriculture road water
3,test_3.jpg,primary clear agriculture road water
4,test_4.jpg,primary clear agriculture road water


In [ ]:
#loading in the weights of the trained model
model2.load_weights('best_model.hdf5')

In [ ]:
# Divide the sample submission file into two splits,
# first test1_df which contains the first 40669 images 
test1_df = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test1_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [ ]:
#initialize imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)


#creating a generator for the images found in the first test image files
test_generator1 = test_image_gen.flow_from_dataframe(dataframe=test1_df, 
                                                directory="../input/planets-dataset/planet/planet/test-jpg", 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=16, 
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

step_test_size1 = int(np.ceil(test_generator1.samples/test_generator1.batch_size))

Found 40669 validated image filenames.


In [ ]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator1.reset()
pred1 = model2.predict(test_generator1, steps = step_test_size1, verbose = 1)

2542/2542 [==============================] - 153s 60ms/step


In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names1 = test_generator1.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
result1.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,clear agriculture primary
4,test_4.jpg,partly_cloudy primary


In [ ]:
#second batch of the test dataset
test2_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
test2_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [ ]:
#creating a generator for the second batch of test image files
test_generator2 = test_image_gen.flow_from_dataframe(dataframe=test2_df, 
                                                directory="../input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=16, 
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

step_test_size2 = int(np.ceil(test_generator2.samples/test_generator2.batch_size))

Found 20522 validated image filenames.


In [ ]:
#we reset the generator to avoid shuffling, then make prediction on the generator
test_generator2.reset()
pred2 = model2.predict(test_generator2, steps = step_test_size2, verbose = 1)

1283/1283 [==============================] - 76s 59ms/step


In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names2 = test_generator2.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names2, 'tags': pred_tags2})
result2.head()

,image_name,tags
0,file_0.jpg,clearprimary
1,file_1.jpg,agriculturepartly_cloudyprimary
2,file_10.jpg,agricultureroadprimary
3,file_100.jpg,clearwaterprimary
4,file_1000.jpg,clearprimary


In [ ]:
#for the final result of the predicted tags for the test images,
# we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

(61191, 2)


,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,partly_cloudy primary
3,test_3.jpg,clear agriculture primary
4,test_4.jpg,partly_cloudy primary


In [ ]:
#we need to remove the .jpg extension from the image_name of the
# last_result because from the sample submission the 
#extension was not there, we added it for easy manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,clear agriculture primary
4,test_4,partly_cloudy primary


In [ ]:
# Finally, we save the result to a csv file using the .to_csv() 
# method and setting the index to false.
last_result.to_csv('submission13.csv', index = False)

NameError: ignored